In [1]:
#~ import subprocess
import espressomd 
from espressomd import reaction_methods
from espressomd import electrostatics
import os, sys, time, random, pprint, tqdm, py3Dmol, shutil, webbrowser
import numpy as np
pprint = pprint.PrettyPrinter(indent = 4).pprint

In [2]:
class base():
    classname = 'base'
    def __init__(self):
        self.time_step = 0.005
        self.samples = {}

        self.Navogadro = 6.022e23 # 1/mol
        self.kT = 1.38064852e-23*300 # J
        self.RT = self.kT * self.Navogadro # J/mol
        # While there are no interactions sigma is an arbitrary number (could be light-year), but it defines the unit of concentration

        self.epsilon = 1.0 # kT --- a measure of energy
        self.sigma = 1.0 # 0.35nm --- a measure of length
        self.unit_of_length = self.sigma_SI = 0.35 # nm
        self.unit = (self.unit_of_length*1e-9)**3*self.Navogadro*1000 # l/mol
        self.punit = self.kT*self.Navogadro/(self.unit/1000) # J/m3 = Pa

        self.tini = time.time()

        self.eq_steps = 10000 # The number of equilibration steps
        self.N_Samples = 100 # The number of samples


    def set_LJ(self):

        if self.sigma:
            lj_eps = self.epsilon
            lj_sig = self.sigma
            #~ lj_cut = 1.12246
            lj_cut = self.sigma*2**(1./6)
            #~ lj_cut = self.sigma_es+1

            P = list(self.TYPES.keys())
            P.remove('Anode-')
            P.remove('Anode0')
            P.remove('Cathode+')
            P.remove('Cathode0')
            pairs = [(P[i], P[j]) for i in range(len(P)) for j in range(i, len(P))]
            #~ print(particles, pairs)
            self.LJ_params = {}
            for pair in pairs:
                ids = [self.TYPES[pair[0]], self.TYPES[pair[1]]]
                self.system.non_bonded_inter[ids[0], ids[1]].lennard_jones.set_params(epsilon = lj_eps, sigma=lj_sig, cutoff=lj_cut, shift="auto")
                self.LJ_params[pair] = self.system.non_bonded_inter[ids[0], ids[1]].lennard_jones.get_params()
                # print(self.system.non_bonded_inter[ids[0], ids[1]].lennard_jones.get_params())
                
            pairs_catplus  = [(P[i], 'Cathode+') for i in range(len(P))]
            pairs_anminus = [(P[i], 'Anode-') for i in range(len(P))]
            pairs_cat0  = [(P[i], 'Cathode0') for i in range(len(P))]
            pairs_an0 = [(P[i], 'Anode0') for i in range(len(P))]
            
            pairs = pairs_catplus+pairs_anminus+ pairs_cat0+pairs_an0
            for pair in pairs:
                ids = [self.TYPES[pair[0]], self.TYPES[pair[1]]]
                self.system.non_bonded_inter[ids[0], ids[1]].lennard_jones.set_params(epsilon = lj_eps, sigma=lj_sig*2, cutoff=2*self.sigma*2**(1./6), shift="auto")
                self.LJ_params[pair] = self.system.non_bonded_inter[ids[0], ids[1]].lennard_jones.get_params()
                # print(self.system.non_bonded_inter[ids[0], ids[1]].lennard_jones.get_params())

    def set_thermostat(self):
        if not hasattr(self, 'seed'):
            self.seed = int(time.time())
        self.system.thermostat.set_langevin(kT=1.0, gamma=1.0, seed = self.seed)
        self.thermostat_params = self.system.thermostat.get_state()

    def minimize_energy(self):
        self.system.integrator.set_steepest_descent(f_max=0, gamma=0.1, max_displacement=0.1)
        self.system.integrator.run(100)   # maximal number of steps
        self.system.integrator.set_vv()  # to switch back to velocity Verlet

    def integrate(self):
        #print ('integration '+str(self.steps['md'])+' steps')
        self.system.integrator.run(steps=self.steps['md'])
        
    def reaction(self):
        
        #print ('reaction '+str(self.steps['re'])+' steps')
        self.RE.reaction(steps = self.steps['re'])

    def getN(self, keys = []):
        # ~ if keys == []: keys = self.keys['re']
        if keys == []: keys = self.TYPES.keys()

        for key in keys:
            self.N[key] = len(self.system.part.select(type=self.TYPES[key]))
            try:                self.samples[key] = np.append(self.samples[key], self.N[key])
            except KeyError:    self.samples[key] = np.array([self.N[key]])
        return self.N

    def energy(self):
        energy = self.system.analysis.energy()['total'] - self.system.analysis.energy()['kinetic']
        try:
            self.samples['energy'] = np.append(self.samples['energy'], energy)
        except KeyError:
            self.samples['energy'] = np.array([energy])
        return energy

    def time(self):
        time = self.system.time
        try:
            self.samples['time'] = np.append(self.samples['time'], time)
        except KeyError:
            self.samples['time'] = np.array([time])
        return time

    def pressure(self):
        pressure = self.system.analysis.pressure()['total']
        try:
            self.samples['pressure'] = np.append(self.samples['pressure'], pressure)
        except KeyError:
            self.samples['pressure'] = np.array([pressure])
        return pressure

    def coords(self):
        self.part = {
                'pos': self.system.part.all().pos,
                'id': self.system.part.all().id,
                'type': self.system.part.all().type,
                'bonds': self.system.part.all().bonds,
                'q': self.system.part.all().q,}
        bonds = []
        BONDS = self.part['bonds']
        for i in range(len(BONDS)):
            bonds.append([i])
            for bond in BONDS[i]:
                 bonds[-1].append(bond[1])
            self.part['bonds'] = bonds

        try:             self.Samples['coords'] = np.append(self.Samples['coords'], self.part)
        except KeyError: self.Samples['coords'] = np.array([self.part])

        return self.part

    def writemol2(self):
        #coords = np.random.choice(self.Samples['coords'])
        coords = self.coords()
        # For reference of mol2 file format see the link https://chemicbook.com/2021/02/20/mol2-file-format-explained-for-beginners-part-2.html
        strings = []

        strings.append("# Name: combgel\n")
        strings.append("@<TRIPOS>MOLECULE\n")
        strings.append(self.name.replace('/','_')+"\n")
        strings.append(str(len(coords['id']))+' '+str(len(coords['bonds'])+7)+'\n')
        strings.append("SMALL\n")
        strings.append("USER_CHARGES\n")
        strings.append("@<TRIPOS>ATOM\n")
        
        coords_pos = coords['pos'] % self.box_l
        for i in range(len(coords['id'])):
            idx = coords['id'][i]
            q = coords['q'][i]
            # name = self.NAMES[coords['type'][i]][0].upper()
            name = self.NAMES[coords['type'][i]]
            coord = coords_pos[i]
            #print("atom {} radius 1 name {} type {}\n".format(idx, typ, typ))
            #radius = 0.5
            #if typ in [self.TYPES['nodes'],self.TYPES['PA'],self.TYPES['PHA']]: radius = 1
            strings.append("{} {} {:3.3f} {:3.3f} {:3.3f} {} {} {} {:3.3f}\n".format(idx, name, coord[0], coord[1], coord[2], name, idx, name, q))
        strings.append("@<TRIPOS>BOND\n")
        
        i = 0
        for b in coords['bonds']:
            if len(b) > 0:
                for j in b[1:]:
                    if np.linalg.norm(coords_pos[b[0]] - coords_pos[j])<2.1:
                        strings.append("{} {} {} 1\n".format(i, b[0], j))
                        i+=1
            #print(i)
        strings[3] = str(len(coords['id']))+' '+str(i)+'\n'
        fp = open(self.fname+'.mol', mode='w+t')
        for s in strings:
            fp.write(s)
        fp.close()
        self.fnamemol = self.fname+'.mol'
        print ('% structure saved in '+self.fnamemol)
        return self.fnamemol
   

In [3]:
class batt(base):
    classname = 'batt'
    def __init__(self):
        self.name = 'batt'
        #~ self.box_l = box_l
        #~ self.volume = box_l**3
        base.__init__(self)
        # Parameters section
        self.val = {
            'Cl':  -1,
            'Na':  +1,
            'Ca':  +2,
            }

        self.sigma = 1.0 # [ sigma ]
        self.lB = 2.0 # Bjerrum length in water [ sigma ]

        try: idx = max(self.TYPES.values())+1
        except AttributeError: idx = 0; self.TYPES = {}

        type_names = ['Cl', 'Na', 'Ca', 'Anode-','Cathode+','Cathode0','Anode0',]
        for type_name in type_names:
            self.TYPES[type_name] = idx; idx+=1

        self.NAMES = {} # the inverse to TYPES dictionary
        for key in self.TYPES.keys():
            val = self.TYPES[key]
            self.NAMES[val] = key


        self.Samples = {}


        self.keys = {} # This is the dictionary for the key values of interest
        self.keys['md'] = ['pressure']
        self.keys['re'] = []
        


        self.steps = {} # The dictionary storing number of steps for each process ie fhen calling integrate, reaction
        self.steps['md'] = 64
        self.steps['re'] = 128

        self.n_samples = {}
        self.n_samples['re'] = 100 # number of samples in one reaction run step
        self.n_samples['md'] = 50 # number of samples in one md step
 
        self.exclusion_radius = 1.0*self.sigma

    def set_electrodes_reaction(self,charge = 0):
        
        if abs(self.pKe) != np.infty: 
            if not hasattr(self,'RE'):
                self.RE = reaction_methods.ReactionEnsemble(
                        kT=1., 
                        exclusion_range=0.5, 
                        seed = np.random.randint(100))
            
            self.Gamma = self.unit*10**(-self.pKe)
            self.RE.add_reaction(gamma = self.Gamma, 
                reactant_types=[self.TYPES['Cathode+'],self.TYPES['Anode-']],
                reactant_coefficients=[1,1],
                product_types=[self.TYPES['Cathode0'],self.TYPES['Anode0']],
                product_coefficients=[1,1],
                default_charges={self.TYPES['Cathode+']:+charge, self.TYPES['Anode-']:-charge, self.TYPES['Cathode0']:0, self.TYPES['Anode0']:0, })

            print ('Setting up electrode reaction , Gamma = '+str(self.Gamma))
            self.RE_params = self.RE.get_status()
            self.RE_params['volume'] = self.RE.get_volume()

    def show(self):
        fnamemol = self.writemol2()
        shutil.copyfile(fnamemol, '/home/kvint/web/'+fnamemol)
        webbrowser.open('127.0.0.1/mol.html?'+fnamemol)
        
        
    def __str__(self):
        self.name = self.classname
        # This generates the suffixes by the dictionary self.p

        self.name += '_box_l'+str(np.round(self.box_l,3))
        if self.lB != 2.0:
            self.name += '_lB'+str(self.lB)
        if self.sigma != 1.0:
            self.name += '_sigma'+str(self.sigma)
        if self.epsilon != 1.0:
            self.name = self.name+'_epsilon'+str(self.epsilon)
        if self.N_Samples != 100: self.name += '_N'+str(self.N_Samples)

        self.fname = 'data/'+self.name
        self.fnameout = self.fname+".out"
        self.fnamepkl = self.fname+".pkl"
        self.fnamepy = self.fname+".py"

        # ~ self.fnamerun     = self.fname+'.run'
        self.fnameqsub    = self.fname+'.qsub'
        self.fnameqsubout = self.fname+'.qsubout'
        self.fnameqsuberr = self.fname+'.qsuberr'

        return self.name

    def update_re_samples(self):
        return self.getN(keys = self.keys['re'])
        
    def update_md_samples(self):
        ''' returns either mindist or pressure(if sigma is not zero)'''
        fl = (not self.sigma)*[self.mindist]+bool(self.sigma)*[self.pressure]
        return [f() for f in fl]

    def set_EL(self,prefactor=1.0, gap_size = 0):
        if self.lB:
            print('\n #### Seting the Electrostatics ####')
            Q = self.system.part.all().q

            tini = time.time();
            if Q.any():
                self.p3m = espressomd.electrostatics.P3M(prefactor=self.lB, accuracy=1e-4)
                if gap_size: 
                    #elc = espressomd.electrostatics.ELC(actor=p3m, gap_size=gap_size, maxPWerror=1e-3)
                    # This simulates the metalic electrodes at z = 0 and z = Lz -h
                    self.elc = espressomd.electrostatics.ELC(actor=self.p3m, gap_size=gap_size, maxPWerror=1e-3, const_pot=True, delta_mid_bot=1)
                    #self.system.actors.add(self.elc)
                    self.p3m_params = self.elc.get_params()
                else:
                    #self.system.actors.add(self.p3m)
                    self.system.electrostatics.solver = self.p3m
                    self.p3m_params = self.p3m.get_params()
                #self.p3m.tune(accuracy=1e-4)
                
                print("Output of p3m captured.")
                uptime = time.time() - tini
                print('Done, uptime = ', uptime)
            else:
                print('No charges --- no P3M, uptime = ')
   
    def py3Dmol(self):
        b = self
        filemol2 = b.writemol2()
        struct = open(filemol2, "r").read()
        z = open(filemol2, "r").read()
        
        box_l = b.box_l
        
        v = py3Dmol.view(width=1000,height=800)
        v.addModel(z,'mol2')
        v.setStyle({}, {'stick': {'color':"red"}, 'sphere': {'color': 'blue', 'radius':0.5}});  # style all atoms */
        v.setStyle({'atom': 'Cathode+'}, {'stick': {'color':"black"}, 'sphere': {'color': '#d59bbb', 'radius':1.0, 'opacity':0.99 }});  #/* style Cathode+ atoms */
        v.setStyle({'atom': 'Anode-'},   {'stick': {'color':"black"}, 'sphere': {'color': '#00bcff', 'radius':1.0, 'opacity':0.9 }});  #/* style Anode- atoms */
        v.setStyle({'atom': 'Cathode0'}, {'stick': {'color':"black"}, 'sphere': {'color': 'grey',    'radius':1.0, 'opacity':0.99 }});  #/* style Cathode0 atoms */
        v.setStyle({'atom': 'Anode0'},   {'stick': {'color':"black"}, 'sphere': {'color': 'grey',    'radius':1.0, 'opacity':0.9 }});  #/* style Anode0 atoms */
        
        v.setStyle({'atom': 'Ca'}, {'stick': {'color':"green"}, 'sphere': {'color': 'green', 'radius':0.8 }});  #/* style Ca atoms */
        v.setStyle({'atom': 'Na'}, {'stick': {'color':"red"}, 'sphere': {'color': 'red', 'radius':0.5 }});  #/* style Na atoms */
        v.setStyle({'atom': 'Cl'}, {'stick': {'color':"blue"}, 'sphere': {'color': 'blue', 'radius':0.5 }});  #/* style Cl atoms */
        
        v.setClickable({},True,
                       '''
                           function(atom,viewer,event,container) {
                               console.log('test', atom)
                               if(!atom.label) {
                                   atom.label = viewer.addLabel(atom.atom + 
                                       ": (" + atom.x+ ", " + atom.y+ ", " + atom.z+ "), q=" + atom.properties.charge, 
                                       { position: atom, backgroundColor: 'darkgreen', backgroundOpacity: 0.8 }
                                       );
                               }}
                       ''',
         
                      )
        #//v.addBox({wireframe: true},{ center: { x: 0, y: 0, z: 0 }, dimensions: { w: box_l, h: box_l, d: box_l }, color: 'magenta' })
        v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': 0, 'z': 0 }, 'end': { 'x': box_l, 'y': 0, 'z': 0 } })
        v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': 0, 'z': 0 }, 'end': { 'x': 0, 'y': box_l, 'z': 0 } })
        v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': 0, 'z': 0 }, 'end': { 'x': 0, 'y': 0, 'z': box_l } })      
        
        v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': box_l, 'z': box_l }, 'end': { 'x': box_l, 'y': box_l, 'z': 0 } })
        v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': box_l, 'z': box_l }, 'end': { 'x': 0, 'y': box_l, 'z': box_l } })
        v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': box_l, 'z': box_l }, 'end': { 'x': box_l, 'y': 0, 'z': box_l } })      
        
        v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': box_l, 'z': 0 }, 'end': { 'x': box_l, 'y': 0, 'z': 0 } })
        v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': box_l, 'z': box_l }, 'end': { 'x': 0, 'y': box_l, 'z': 0 } })
        v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': 0, 'z': box_l }, 'end': { 'x': 0, 'y': 0, 'z': box_l } })
        
        
        v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': box_l, 'z': 0 }, 'end': { 'x': box_l, 'y': box_l, 'z': 0 } })
        v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': 0, 'z': box_l }, 'end': { 'x': 0, 'y': box_l, 'z': box_l } })
        v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': 0, 'z': 0 }, 'end': { 'x': box_l, 'y': 0, 'z': box_l } })          
                                 
        
        v.zoomTo();                                      #/* set camera */
        #v.render();                                      #/* render scene */
        v.zoom(1.2, 1000);                               #/* sliht zoom */
        v.show()
        #v.spin()
   


In [4]:
b = batt()
cs = 0.1 # mol/l --- salinity of electrolyte
rho = 0.5 # density of charges on the electrode


b.qsu = 1 # d


b.N_Samples = 100 # A number of samples generated by sample() function
b.sigma = 1.0
#s.lB = 2.0
b.lB = 2.0


cna = cs


nna = 200 # A number of sodium ions
nca = int(np.ceil(nna*0.114)) # A number of calcium ions (a fraction of nna)
ncl = nna + 2*nca #number of chloride ions (neutralizing Na and Ca)

cs_es = cs*b.unit # A desired density of the Na ions in a simulation box 
b.box_l = (nna/cs_es)**(1./3) # The zize of the box which guarantee the desired cs_es
b.volume = b.box_l**3 #Volime of the box
print ('box_l = ', b.box_l)

b.system = espressomd.System(box_l = 3*[b.box_l])

type_suplus = b.TYPES['Cathode+']
type_suminus = b.TYPES['Anode-']
type_cl = b.TYPES['Cl']
type_na = b.TYPES['Na']
type_ca = b.TYPES['Ca']


# Add surface
Nsu_ = int(np.ceil(b.box_l/b.sigma)) # number of beads in one dimension
Nsu = Nsu_**2                        # number of beads in the electrode
qsu = int(rho*b.box_l**2) / Nsu      # the valence of a bead of electrode
# qsu = -1
b.gap_size = b.box_l/3

# First surface
for i in range(Nsu_):
    for j in range(Nsu_):
        pos=[i,j,b.gap_size]
        b.system.part.add(pos=pos, type=b.TYPES['Cathode+'], q=qsu, fix = 3*[True])

# Second surface
for i in range(Nsu_):
    for j in range(Nsu_):
        pos=[i,j, 2 * b.gap_size]
        b.system.part.add(pos=pos, type=b.TYPES['Anode-'], q=-qsu, fix = 3*[True])

b.qsu = qsu

for i in range(nna):
    pos = np.random.rand(3)*b.box_l*[1,1,1/3] + [0,0,b.box_l/3]
    b.system.part.add(pos=pos, type=type_na, q=1)

for i in range(ncl):
    pos = np.random.rand(3)*b.box_l*[1,1,1/3] + [0,0,b.box_l/3]
    b.system.part.add(pos=pos, type=type_cl, q=-1)

for i in range(nca):
    pos = np.random.rand(3)*b.box_l*[1,1,1/3] + [0,0,b.box_l/3]
    b.system.part.add(pos=pos, type=type_ca, q=2)

b.system.time_step = b.time_step
b.system.cell_system.skin = 0.4

b.integrate()
b.set_LJ()
print(b)


box_l =  42.628007856291234
batt_box_l42.628


In [5]:
b.pKe = 10 # pK for the reaction between electrodes
b.set_electrodes_reaction(charge = b.qsu)
b.RE.reactions


Setting up electrode reaction , Gamma = 2.5819325e-12


In [6]:
#b.RE.delete_reaction(reaction_id=0)

In [7]:



if b.lB: b.set_EL(gap_size = 0)
#wall0 = espressomd.shapes.Wall(dist=0, normal=[1, 0.0, 0])
#wall1 = espressomd.shapes.Wall(dist=s.gap_size, normal=[0.0, 0.0, 1])
# s.system.constraints.add(shape=wall0, particle_type=0)
# s.system.constraints.add(shape=wall1, particle_type=0)

b.minimize_energy()
b.integrate()




 #### Seting the Electrostatics ####
Output of p3m captured.
Done, uptime =  9.37740421295166
CoulombP3M tune parameters: Accuracy goal = 1.00000e-04 prefactor = 2.00000e+00
System: box_l = 4.26280e+01 # charged part = 4167 Sum[q_i^2] = 1.42979e+03
mesh cao r_cut_iL    alpha_L     err       rs_err    ks_err    time [ms]
16   7   2.91304e-01 9.18098e+00 9.904e-05 7.071e-05 6.934e-05 87.22   
16   6   3.05677e-01 8.73453e+00 9.893e-05 7.071e-05 6.919e-05 92.19   
20   7   2.37253e-01 1.13531e+01 9.918e-05 7.071e-05 6.954e-05 56.56   
20   6   2.49201e-01 1.07905e+01 9.903e-05 7.071e-05 6.933e-05 58.93   
26   7   1.86281e-01 1.45797e+01 9.929e-05 7.071e-05 6.970e-05 34.21   
26   6   1.96475e-01 1.37983e+01 9.790e-05 7.071e-05 6.771e-05 38.21   
30   7   1.63723e-01 1.66609e+01 9.769e-05 7.071e-05 6.741e-05 28.33   
30   6   1.71728e-01 1.58589e+01 9.958e-05 7.071e-05 7.012e-05 31.23   
34   7   1.45816e-01 1.87797e+01 9.784e-05 7.071e-05 6.763e-05 27.47   
34   6   1.53491e-01 1.78101e

In [8]:
b.RE.reactions


% structure saved in data/batt_box_l42.628.mol


In [10]:
z = open(filemol2, "r").read()
print (z[:1000])

# Name: combgel
@<TRIPOS>MOLECULE
batt_box_l42.628
4167 0
SMALL
USER_CHARGES
@<TRIPOS>ATOM
0 Cathode+ 0.000 0.000 14.209 Cathode+ 0 Cathode+ 0.491
1 Cathode+ 0.000 1.000 14.209 Cathode+ 1 Cathode+ 0.491
2 Cathode+ 0.000 2.000 14.209 Cathode+ 2 Cathode+ 0.491
3 Cathode+ 0.000 3.000 14.209 Cathode+ 3 Cathode+ 0.491
4 Cathode+ 0.000 4.000 14.209 Cathode+ 4 Cathode+ 0.491
5 Cathode+ 0.000 5.000 14.209 Cathode+ 5 Cathode+ 0.491
6 Cathode+ 0.000 6.000 14.209 Cathode+ 6 Cathode+ 0.491
7 Cathode+ 0.000 7.000 14.209 Cathode+ 7 Cathode+ 0.491
8 Cathode+ 0.000 8.000 14.209 Cathode+ 8 Cathode+ 0.491
9 Cathode+ 0.000 9.000 14.209 Cathode+ 9 Cathode+ 0.491
10 Cathode+ 0.000 10.000 14.209 Cathode+ 10 Cathode+ 0.491
11 Cathode+ 0.000 11.000 14.209 Cathode+ 11 Cathode+ 0.491
12 Cathode+ 0.000 12.000 14.209 Cathode+ 12 Cathode+ 0.491
13 Cathode+ 0.000 13.000 14.209 Cathode+ 13 Cathode+ 0.491
14 Cathode+ 0.000 14.000 14.209 Cathode+ 14 Cathode+ 0.491
15 Cathode+ 0.000 15.000 14.209 Cathode+ 15 Cathode+ 0

In [11]:
b.box_l

42.628007856291234

In [12]:

filemol2 = b.writemol2()
struct = open(filemol2, "r").read()
z = open(filemol2, "r").read()

box_l = b.box_l

v = py3Dmol.view(width=1000,height=800)
v.addModel(z,'mol2')
v.setStyle({}, {'stick': {'color':"red"}, 'sphere': {'color': 'blue', 'radius':0.5}});  # style all atoms */
v.setStyle({'atom': 'Cathode+'}, {'stick': {'color':"black"}, 'sphere': {'color': '#d59bbb', 'radius':1.0, 'opacity':0.99 }});  #/* style Cathode+ atoms */
v.setStyle({'atom': 'Anode-'},   {'stick': {'color':"black"}, 'sphere': {'color': '#00bcff', 'radius':1.0, 'opacity':0.9 }});  #/* style Anode- atoms */
v.setStyle({'atom': 'Cathode0'}, {'stick': {'color':"black"}, 'sphere': {'color': 'grey',    'radius':1.0, 'opacity':0.99 }});  #/* style Cathode0 atoms */
v.setStyle({'atom': 'Anode0'},   {'stick': {'color':"black"}, 'sphere': {'color': 'grey',    'radius':1.0, 'opacity':0.9 }});  #/* style Anode0 atoms */

v.setStyle({'atom': 'Ca'}, {'stick': {'color':"green"}, 'sphere': {'color': 'green', 'radius':0.8 }});  #/* style Ca atoms */
v.setStyle({'atom': 'Na'}, {'stick': {'color':"red"}, 'sphere': {'color': 'red', 'radius':0.5 }});  #/* style Na atoms */
v.setStyle({'atom': 'Cl'}, {'stick': {'color':"blue"}, 'sphere': {'color': 'blue', 'radius':0.5 }});  #/* style Cl atoms */

v.setClickable({},True,
               '''
                   function(atom,viewer,event,container) {
                       console.log('test', atom)
                       if(!atom.label) {
                           atom.label = viewer.addLabel(atom.atom + 
                               ": (" + atom.x+ ", " + atom.y+ ", " + atom.z+ "), q=" + atom.properties.charge, 
                               { position: atom, backgroundColor: 'darkgreen', backgroundOpacity: 0.8 }
                               );
                       }}
               ''',
 
              )
#//v.addBox({wireframe: true},{ center: { x: 0, y: 0, z: 0 }, dimensions: { w: box_l, h: box_l, d: box_l }, color: 'magenta' })
v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': 0, 'z': 0 }, 'end': { 'x': box_l, 'y': 0, 'z': 0 } })
v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': 0, 'z': 0 }, 'end': { 'x': 0, 'y': box_l, 'z': 0 } })
v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': 0, 'z': 0 }, 'end': { 'x': 0, 'y': 0, 'z': box_l } })      

v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': box_l, 'z': box_l }, 'end': { 'x': box_l, 'y': box_l, 'z': 0 } })
v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': box_l, 'z': box_l }, 'end': { 'x': 0, 'y': box_l, 'z': box_l } })
v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': box_l, 'z': box_l }, 'end': { 'x': box_l, 'y': 0, 'z': box_l } })      

v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': box_l, 'z': 0 }, 'end': { 'x': box_l, 'y': 0, 'z': 0 } })
v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': box_l, 'z': box_l }, 'end': { 'x': 0, 'y': box_l, 'z': 0 } })
v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': 0, 'z': box_l }, 'end': { 'x': 0, 'y': 0, 'z': box_l } })


v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': box_l, 'z': 0 }, 'end': { 'x': box_l, 'y': box_l, 'z': 0 } })
v.addLine({ 'color': 'black', 'start': { 'x': 0, 'y': 0, 'z': box_l }, 'end': { 'x': 0, 'y': box_l, 'z': box_l } })
v.addLine({ 'color': 'black', 'start': { 'x': box_l, 'y': 0, 'z': 0 }, 'end': { 'x': box_l, 'y': 0, 'z': box_l } })          
                         

v.zoomTo();                                      #/* set camera */
#v.render();                                      #/* render scene */
v.zoom(1.2, 1000);                               #/* sliht zoom */
v.show()
#v.spin()


% structure saved in data/batt_box_l42.628.mol


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
help(v.add)


Help on function makejs in module py3Dmol:

makejs(*args, **kwargs)



In [14]:
b.RE.delete_reaction(reaction_id=0)
b.pKe = 10 # pK for the reaction between electrodes
b.set_electrodes_reaction(charge = b.qsu)
b.RE.reactions
for i in (pbar := tqdm.trange(10)): 
    # b.integrate()
    
    
    b.reaction()
    b.integrate()
    N0 = len(b.system.part.select(type = b.TYPES['Cathode0']))+len(b.system.part.select(type = b.TYPES['Anode0']))
    Nplus = len(b.system.part.select(type = b.TYPES['Cathode+']))
    Nminus = len(b.system.part.select(type = b.TYPES['Anode-']))
    pbar.set_description("\\alpha = {:.2f}".format((Nplus+Nminus)/(N0+Nplus+Nminus)))
    #print ('N0/N =', N0/(N0+Nplus+Nminus))
        


  0%|                                                                                                                                                                                | 0/10 [00:00<?, ?it/s]

Setting up electrode reaction , Gamma = 2.5819325e-12


\alpha = 0.67: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:02<00:00,  6.23s/it]


In [15]:
b.py3Dmol()

% structure saved in data/batt_box_l42.628.mol


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
b.RE.delete_reaction(reaction_id=0)
b.pKe = -10 # pK for the reaction between electrodes
b.set_electrodes_reaction(charge = b.qsu)
b.RE.reactions
for i in (pbar := tqdm.trange(50)): 
    # b.integrate()
    
    
    b.reaction()
    b.integrate()
    N0 = len(b.system.part.select(type = b.TYPES['Cathode0']))+len(b.system.part.select(type = b.TYPES['Anode0']))
    Nplus = len(b.system.part.select(type = b.TYPES['Cathode+']))
    Nminus = len(b.system.part.select(type = b.TYPES['Anode-']))
    pbar.set_description("\\alpha = {:.2f}".format((Nplus+Nminus)/(N0+Nplus+Nminus)))
    #print ('N0/N =', N0/(N0+Nplus+Nminus))
        
b.py3Dmol()

  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

Setting up electrode reaction , Gamma = 258193249.99999997


\alpha = 0.50:  10%|███████████████▎                                                                                                                                         | 5/50 [00:27<04:09,  5.55s/it]

In [ ]:
b.RE.delete_reaction(reaction_id=0)
b.pKe = 0 # pK for the reaction between electrodes
b.set_electrodes_reaction(charge = b.qsu)
b.RE.reactions
for i in (pbar := tqdm.trange(50)): 
    # b.integrate()
    
    
    b.reaction()
    b.integrate()
    N0 = len(b.system.part.select(type = b.TYPES['Cathode0']))+len(b.system.part.select(type = b.TYPES['Anode0']))
    Nplus = len(b.system.part.select(type = b.TYPES['Cathode+']))
    Nminus = len(b.system.part.select(type = b.TYPES['Anode-']))
    pbar.set_description("\\alpha = {:.2f}".format((Nplus+Nminus)/(N0+Nplus+Nminus)))
    #print ('N0/N =', N0/(N0+Nplus+Nminus))
b.py3Dmol()

In [ ]:
b.RE.delete_reaction(reaction_id=0)
b.pKe = 10 # pK for the reaction between electrodes
b.set_electrodes_reaction(charge = b.qsu)
b.RE.reactions
for i in (pbar := tqdm.trange(50)): 
    # b.integrate()
    
    
    b.reaction()
    b.integrate()
    N0 = len(b.system.part.select(type = b.TYPES['Cathode0']))+len(b.system.part.select(type = b.TYPES['Anode0']))
    Nplus = len(b.system.part.select(type = b.TYPES['Cathode+']))
    Nminus = len(b.system.part.select(type = b.TYPES['Anode-']))
    pbar.set_description("\\alpha = {:.2f}".format((Nplus+Nminus)/(N0+Nplus+Nminus)))
    #print ('N0/N =', N0/(N0+Nplus+Nminus))
b.py3Dmol()

In [ ]:
b.RE.delete_reaction(reaction_id=0)
b.pKe = 20 # pK for the reaction between electrodes
b.set_electrodes_reaction(charge = b.qsu)
b.RE.reactions
for i in (pbar := tqdm.trange(50)): 
    # b.integrate()
    
    
    b.reaction()
    b.integrate()
    N0 = len(b.system.part.select(type = b.TYPES['Cathode0']))+len(b.system.part.select(type = b.TYPES['Anode0']))
    Nplus = len(b.system.part.select(type = b.TYPES['Cathode+']))
    Nminus = len(b.system.part.select(type = b.TYPES['Anode-']))
    pbar.set_description("\\alpha = {:.2f}".format((Nplus+Nminus)/(N0+Nplus+Nminus)))
    #print ('N0/N =', N0/(N0+Nplus+Nminus))
b.py3Dmol()